In [1]:
%matplotlib qt
import os
import broadbean as bb
import numpy as np
import pandas as pd
import panel as pn
import tempfile
import qcodes as qc
import matplotlib.pyplot as plt
from sequencebuilder.alazar_config import alazarconfig
from sequencebuilder.refrenceinstrument import AlazarInstrument
from broadbean.plotting import plotter
from qcodes.utils.dataset.doNd import do0d, do1d
from sequencebuilder.spinbuilder import AWGController
from qcodes import initialise_or_create_database_at, \
    load_or_create_experiment, Measurement, Parameter, \
    Station
from sequencebuilder.spinbuilder import SpinBuilder
from pulsequantum.elem_from_plot import elem_on_plot

pn.extension('tabulator')
ramp = bb.PulseAtoms.ramp

In [2]:
db_path = os.path.join(tempfile.gettempdir(),
                       'data_access_example.db')
initialise_or_create_database_at(db_path)

experiment = load_or_create_experiment(
    experiment_name='alazar',
    sample_name='alazar-sample')
SC = qc.Station(config_file="C:\\Users\\Farzad\\Documents\\Rasmus\\QCoDeS-experiment\\ConfigRBC.yaml")
AWGA = SC.load_instrument('AWGA')
seqbuild = AWGController(name = 'seqbuild', awg=AWGA)
SC.add_component(seqbuild)

Connected to: TEKTRONIX AWG5014C (serial:B051370, firmware:SCPI:99.0 FW:4.6.0.7) in 0.75s


'seqbuild'

In [ ]:
alazar = SC.load_instrument('alazar')
alazarconfig(alazar, seqmode=True,external_clock=False)
alazar_ctrl = SC.load_instrument('alazar_ctrl')
channelA_oneD = SC.load_instrument('channelA_oneD',parent=alazar_ctrl)
alazar_ctrl.channels.append(channelA_oneD)


In [ ]:
seqbuild.ch_x = 1
seqbuild.ch_y = 2
seqbuild.divider = {'ch1': 0.5, 'ch2': 0.5} 

In [ ]:
seqbuild.spinfunnel()


In [ ]:
seqbuild.divider

In [9]:
df_widget = pn.widgets.Tabulator(seqbuild.df)
df_widget

Tabulator(value=       name  time  type   ...)

In [6]:
seqbuild.seq_from_df()
seqbuild.seq.plot_elem_nr(1)
seqbuild.seq.plot_elem_nr(100)

I was here
I was here


In [ ]:
x,y,ramp = elem_on_plot(33)

In [ ]:
len(x)

In [ ]:
seqbuild.df_from_list(x,y,ramp)

In [ ]:
seqbuild.update_seq_from_list(x,y,ramp)

In [ ]:
seqbuild.seq.plot_elem_nr(1)

In [ ]:
seqbuild.uploadToAWG()

In [ ]:
seqbuild.awg.ch1_state(1)
seqbuild.awg.ch2_state(1)

In [ ]:
print(seqbuild.awg.ch1_state())
print(seqbuild.awg.ch2_state())


In [ ]:
seqbuild.awg.run()

In [ ]:
seqbuild.awg.get_state()

In [ ]:
alazar_ctrl.int_time(0.9e-6)
alazar_ctrl.int_delay(1e-7)
channelA_oneD.num_averages(300)
channelA_oneD.records_per_buffer(200)
channelA_oneD.prepare_channel()

#channelA_oneD.data.get()

In [ ]:
do0d(channelA_oneD.data, do_plot=True)

In [ ]:
testbla = AlazarInstrument('bla',channelA_oneD.data.get,100)

In [ ]:
do0d(testbla.datarefrence, do_plot=True)

In [ ]:
do1d(testbla.V_dc, 0, 1, 10, 0.01,testbla.datarefrence, do_plot=True)

In [7]:
seqbuild.df.to_csv('test.csv')

In [8]:
seqbuild.df = pd.read_csv('test.csv', index_col=0)